In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
df = pd.read_csv('../input/your-employees-leave-you-dataset/Train.csv')

In [ ]:
df.head(50)

In [ ]:
df.shape

In [ ]:
df['VAR7'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
# function to fill null/NaN values in categorical columns
def categorical_null(columns):
    for column in columns:
        df[column] = df[column].fillna(df[column].mode()[0])
        
# function to fill null/NaN values in integer value columns
def integer_null(columns):
    for column in columns:
        df[column] = df[column].fillna(df[column].mean())

In [ ]:
categorical_null_columns = ['Pay_Scale', 'Work_Life_balance', 'VAR2', 'VAR4']
integer_value_null_columns = ['Age', 'Time_of_service']

categorical_null(categorical_null_columns)
integer_null(integer_value_null_columns)

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='coolwarm')

In [ ]:
df.loc[:, df.isnull().any()].head()

In [ ]:
df.info()

In [ ]:
# combine test data
test_df = pd.read_csv('../input/formulatedtest3/formulatedtest(2).csv')

In [ ]:
test_df.head()

In [ ]:
final_df = pd.concat([df, test_df], axis=0)

In [ ]:
final_df.tail()

In [ ]:
final_df.shape

In [ ]:
def category_onehot_multcols(multcolumns):
    #df_final = final_df
    i = 0
    for fields in multcolumns:
        print(fields)
        df1 = pd.get_dummies(final_df[fields], drop_first=True)
        
        final_df.drop([fields], axis=1, inplace=True)
        if i==0:
            df_final = df1.copy()
        else:
            df_final = pd.concat([df_final, df1], axis=1)
        i=i+1
    
    df_final = pd.concat([final_df, df_final], axis=1)
    
    return df_final

In [ ]:
final_df.drop(['Employee_ID'], axis=1, inplace=True)

In [ ]:
columns = ['Gender', 'Relationship_Status', 'Hometown', 'Unit', 'Decision_skill_possess',
           'Travel_Rate', 'Compensation_and_Benefits', 
           'VAR2', 'VAR3', 'VAR4', 'VAR6']

In [ ]:
len(columns)

In [ ]:
final_df = category_onehot_multcols(columns)

In [ ]:
final_df.head()

In [ ]:
final_df.shape

In [ ]:
# divide train and test data
df_Train = final_df.iloc[:7000, :]
df_Test = final_df.iloc[7000:, :]

In [ ]:
print(df_Train.shape)
print(df_Test.shape)

In [ ]:
X_train =df_Train.drop(['Attrition_rate'], axis=1)
y_train = df_Train['Attrition_rate']

In [ ]:
X_train.head()

In [ ]:
import xgboost
regressor = xgboost.XGBRegressor()

In [ ]:
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster = ['gbtree', 'gblinear']
learning_rate = [0.05,0.1,0.15,0.20]
min_child_weight = [1,2,3,4]
base_score = [0.25, 0.5, 0.75, 1]

# define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators' : n_estimators,
    'max_depth' : max_depth,
    'learning_rate' : learning_rate,
    'min_child_weight' : min_child_weight,
    'booster' : booster,
    'base_score' : base_score
}

In [ ]:
# set up the random search with 4-fold cross validation
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=regressor,
                              param_distributions=hyperparameter_grid,
                              cv=5, n_iter=50,
                              scoring = 'neg_mean_absolute_error', n_jobs=4,
                              verbose=5,
                              return_train_score = True,
                              random_state  = 42)

In [ ]:
random_cv.fit(X_train, y_train)

In [ ]:
random_cv.best_estimator_

In [ ]:
regressor = xgboost.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=None, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
df_Test.shape

In [ ]:
df_Test.drop(['Attrition_rate'], axis=1, inplace=True)

In [ ]:
df_Test.shape

In [ ]:
df_Test.head()

In [ ]:
y_pred = regressor.predict(df_Test)
y_pred

In [ ]:
pred = pd.DataFrame(y_pred)

datasets = pd.concat([test_df['Employee_ID'], pred], axis=1)
datasets.columns=['Employee_ID', 'Attrition_rate']
datasets.to_csv('sample_submission.csv', index=False)



In [ ]:
datasets